In [1]:
import base64
import collections
import datetime
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import string
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
from bokeh import io, plotting, models, layouts, palettes
import ipywidgets
import xgboost as xgb

np.random.seed(1337)

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

%matplotlib inline

sns.set(font_scale=1.0)
mpl.rcParams['figure.figsize'] = 10, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

plotting.output_notebook()

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python2.7/dist-pac

Loading BokehJS ...

In [2]:
import utils as u

# Load the data

In [3]:
t = pd.read_csv('input/train_2016_v2.csv', low_memory=False)
print len(t)

90275


In [4]:
t.sample(10)

,parcelid,logerror,transactiondate
3501,12493455,0.0325,2016-01-19
34228,12201399,0.0247,2016-05-11
5100,14418239,0.1098,2016-01-26
70141,11888087,-0.0222,2016-08-26
55240,12026764,0.0516,2016-07-13
23987,12948116,-0.0121,2016-04-08
62117,12054831,-0.0253,2016-08-01
76222,14458924,0.0080,2016-09-15
40604,12890059,0.0129,2016-05-31
73715,11745152,-0.0161,2016-09-07


In [5]:
p = pd.read_csv('input/properties_2016.csv', low_memory=False)
print len(p)

2985217


In [6]:
p.sample(10)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
2675901,17060405,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,1478.0,1478.0,1478.0,NaN,NaN,1478.0,NaN,6111.0,1.0,2.0,4.0,960.0,NaN,NaN,34390861.0,-119299672.0,10018.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,40110.0,2061.0,NaN,97098.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,1.0,NaN,157000.0,505000.0,2015.0,348000.0,6036.02,NaN,NaN,6.111001e+13
2133122,12173503,NaN,NaN,NaN,1.0,2.0,NaN,7.0,1.0,NaN,NaN,1801.0,1801.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,2.0,34173097.0,-118098845.0,6958.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LCR175,6.037461e+07,NaN,3101.0,NaN,96292.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1939.0,NaN,NaN,51840.0,84857.0,2015.0,33017.0,1345.60,NaN,NaN,6.037461e+13
1912651,12793767,NaN,NaN,NaN,1.0,3.0,NaN,7.0,1.0,NaN,NaN,1068.0,1068.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,7.0,33943646.0,-118030368.0,5907.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LCR1YY,6.037503e+07,118225.0,3101.0,NaN,96173.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,95062.0,362092.0,2015.0,267030.0,5822.02,NaN,NaN,6.037503e+13
101409,14345567,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,NaN,1424.0,1424.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,426.0,NaN,NaN,33707074.0,-117773602.0,3500.0,NaN,NaN,NaN,NaN,NaN,122,261.0,NaN,6.059053e+07,52650.0,1286.0,276486.0,96954.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,1984.0,2.0,NaN,147385.0,337670.0,2015.0,190285.0,3604.18,NaN,NaN,6.059053e+13
2053208,17240611,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,NaN,1489.0,2488.0,2488.0,NaN,NaN,1489.0,NaN,6111.0,1.0,3.0,2.0,581.0,NaN,NaN,34290095.0,-118674274.0,8060.0,NaN,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111008e+07,27110.0,2061.0,NaN,97116.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1986.0,2.0,NaN,343441.0,572399.0,2015.0,228958.0,6690.54,NaN,NaN,6.111008e+13
1209699,11858524,NaN,NaN,NaN,4.0,4.0,NaN,4.0,4.0,NaN,NaN,2814.0,2814.0,NaN,NaN,NaN,NaN,6037.0,NaN,4.0,NaN,NaN,NaN,2.0,34025987.0,-118094357.0,8352.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,MNR1*,6.037530e+07,25953.0,3101.0,NaN,96192.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1966.0,NaN,NaN,157616.0,230644.0,2015.0,73028.0,3790.38,NaN,NaN,6.037530e+13
866552,11549824,NaN,NaN,NaN,1.0,2.0,NaN,7.0,1.0,NaN,NaN,948.0,948.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,7.0,34002630.0,-118465808.0,4401.0,NaN,NaN,NaN,NaN,NaN,0100,261.0,LAR1,6.037273e+07,12447.0,3101.0,21056.0,96127.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1946.0,NaN,NaN,50903.0,254543.0,2015.0,203640.0,3147.21,NaN,NaN,6.037273e+13
2555813,14684034,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,NaN,1128.0,1128.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,1.0,0.0,NaN,NaN,33640426.0,-117588669.0,NaN,NaN,NaN,NaN,NaN,NaN,34,266.0,NaN,6.059032e+07,15554.0,1286.0,NaN,96993.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1995.0,NaN,NaN,149437.0,266500.0,2015.0,117063.0,3419.14,NaN,NaN,6.059032e+13
104526,17128055,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,1664.0,3099.0,3099.0,NaN,NaN,1664.0,NaN,6111.0,NaN,3.0,2.0,685.0,NaN,NaN,34232751.0,-119055662.0,7697.0,NaN,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111005e+07,51239.0,2061.0,NaN,97089.0,9.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,2.0,NaN,274663.0,457769.0,2015.0,183106.0,4992.98,NaN,NaN,6.111005e+13
2295229,11892761,NaN,NaN,NaN,3.0,7.0,NaN,4.

In [7]:
d = pd.merge(t, p, on='parcelid', how='outer')
del t
del p
print len(d)
i = 0
for c in d.columns:
    print '{0} {1:30s} {2}'.format(i, c, d[c].dtype)
    i += 1

2985342
0 parcelid                       int64
1 logerror                       float64
2 transactiondate                object
3 airconditioningtypeid          float64
4 architecturalstyletypeid       float64
5 basementsqft                   float64
6 bathroomcnt                    float64
7 bedroomcnt                     float64
8 buildingclasstypeid            float64
9 buildingqualitytypeid          float64
10 calculatedbathnbr              float64
11 decktypeid                     float64
12 finishedfloor1squarefeet       float64
13 calculatedfinishedsquarefeet   float64
14 finishedsquarefeet12           float64
15 finishedsquarefeet13           float64
16 finishedsquarefeet15           float64
17 finishedsquarefeet50           float64
18 finishedsquarefeet6            float64
19 fips                           float64
20 fireplacecnt                   float64
21 fullbathcnt                    float64
22 garagecarcnt                   float64
23 garagetotalsqft                float

In [8]:
len(d.parcelid.unique())

2985217

In [9]:
u.summary(d.parcelid)

------------------------------
name:             parcelid
dtype:            int64
num. rows:        2985342
N/A count:        0
N/A ratio:        0.0
distinct values:  2985217
------------------------------ 
top 3 values:
11842707    3
11513960    2
10821829    2
Name: parcelid, dtype: int64
------------------------------ 
bottom 3 values:
10918278    1
10912133    1
12582912    1
Name: parcelid, dtype: int64
------------------------------
count    2.985342e+06
mean     1.332584e+07
std      7.909808e+06
min      1.071172e+07
25%      1.164370e+07
50%      1.254509e+07
75%      1.409712e+07
max      1.696019e+08
Name: parcelid, dtype: float64
------------------------------
